In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch
!pip install numpy
!pip install efficientnet_pytorch

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
import torchvision.models as models
import time
import datetime
from efficientnet_pytorch import EfficientNet

In [4]:
# 모델 학습에 사용할 연산 장치를 설정합니다..
if torch.cuda.is_available():
  DEVICE = torch.device('cuda')
else:
  DEVICE = torch.device('cpu')

print(DEVICE)

cuda


In [5]:
# 모델학습에 사용할 하이퍼 파라미터를 설정합니다.
batch_size = 32
epochs = 5

In [6]:
path = '/content/drive/MyDrive/data/ComputerVision'

In [7]:
# CIFAR-10 Train 데이터를 받아옵니다.
train_dataset = datasets.CIFAR10(root=path,
                                 train = True,
                                 download = True,
                                 transform = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]))
# CIFAR-10 Test 데이터를 받아옵니다.
test_dataset = datasets.CIFAR10(root=path,
                                train = False,
                                download = True,
                                transform = transforms.Compose([
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]))

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# 받아온 CIFA-10 데이터를 DataLoader를 이용해 batch_size 차원을 추가해서 변환합니다.
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size= batch_size, shuffle=True)

In [9]:
for x, y in train_loader:
  print('변환 후 데이터 Shape : ', x.size())
  print('레이블 데이터 수 : ', y.size())
  break

변환 후 데이터 Shape :  torch.Size([32, 3, 32, 32])
레이블 데이터 수 :  torch.Size([32])


In [10]:
# CNN 모델을 정의합니다.
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    # 첫번째 Convolution Layer를 추가합니다.
    # 위에서 확인했듯이 이미지 파일의 channel이 1이므로 처음 입력값에 동일하게 1로 설정합니다
    # 나머지 파라미터는 적절한 값을 주어 구조를 설계합니다.
    self.conv1 = nn.Conv2d(
        in_channels = 3,
        out_channels = 8,
        kernel_size = 3,
        padding = 1
    )

    # 두번째 Convolution Layer를 추가합니다.
    self.conv2 = nn.Conv2d(
        in_channels = 8,
        out_channels = 16,
        kernel_size = 3,
        padding = 1
    )

    # 주어진 데이터의 Feature size를 줄여주는 ooling Layer를 추가합니다. 
    self.pool = nn.MaxPool2d(
        kernel_size = 2
    )

    # 이미지 분류를 위한 Fully Connected Layer를 추가합니다
    self.fc1 = nn.Linear(8 * 8 * 16, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 32)
    self.fc4 = nn.Linear(32, 10)
    

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = self.pool(x)

    # 이미지 데이터를 Fully Connected Layer의 Input으로 넣기 위해 1차원으로 Flatten 시켜줍니다.
    x = x.view(-1, 8 * 8 * 16)

    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)
    x = F.log_softmax(x)
    return x

In [11]:
# GPU에 모델을 할당합니다.
model = CNN().to(DEVICE)

# 학습에 사용할 최적화 함수와 학숩률을 설정합니다.
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# 학습에 사용할 비용함수를 정의합니다.
criterion = nn.CrossEntropyLoss()

print(model)

CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=10, bias=True)
)


In [12]:
# 모델을 학습할 때 사용할 함수를 정의합니다.
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [13]:
# 모델을 평가할 때 사용할 함수를 정의합니다.
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [14]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.297692


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.929282
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.970137
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.753052
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.445458
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.165226
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.558990
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.517837

[EPOCH: 1], 	Test Loss: 0.0417, 	Test Accuracy: 51.48 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.531181
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.956446
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.190615
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.116204
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 1.390048
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 1.268612
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 1.188798
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 1.175460

[EPOCH: 2], 	Test Loss: 0.0364, 	Test Accuracy: 58.58 % 

Train Epoch: 3 [0/50000 (0%)]	Train Loss: 0.91431

In [15]:
# torchvision에서 Resnet 모델을 불러옵니다.
model = models.resnet34(pretrained = False)

# Fully Connected Layer의 출력층 개수를 데이터에 맞게 수정합니다.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

# 모델을 연산장치에 할당합니다.
model = model.cuda()

In [16]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.914049
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.668450
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.527676
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.453699
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.271796
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 1.338886
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 1.631921
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 1.681336

[EPOCH: 1], 	Test Loss: 0.0409, 	Test Accuracy: 53.62 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 1.255712
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.964629
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 1.350717
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 1.205843
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.766252
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.812900
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.805487
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 0.943583

[EPOCH: 2], 	Test Loss: 0.0346, 	Test Accuracy: 63.79 % 

In [18]:
# EfficientNet-B4 모델을 불러옵니다.
model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=10)

model = model.cuda()

Loaded pretrained weights for efficientnet-b4


In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()

print(model)

EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        48, 48, kernel_size=(3, 3), stride=[1, 1], groups=48, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm2d(48, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        48, 12, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        12, 48, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        48, 24, kernel_siz

In [20]:
start = time.time()
for epoch in range(1, epochs + 1):
    train(model, train_loader, optimizer, log_interval = 200)
    test_loss, test_accuracy = evaluate(model, test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))
    
sec = time.time()-start
times = str(datetime.timedelta(seconds=sec)).split(".")
times = times[0]
print(times)    

Train Epoch: 1 [0/50000 (0%)]	Train Loss: 2.314797
Train Epoch: 1 [6400/50000 (13%)]	Train Loss: 1.193741
Train Epoch: 1 [12800/50000 (26%)]	Train Loss: 1.223294
Train Epoch: 1 [19200/50000 (38%)]	Train Loss: 1.180920
Train Epoch: 1 [25600/50000 (51%)]	Train Loss: 1.011616
Train Epoch: 1 [32000/50000 (64%)]	Train Loss: 0.753591
Train Epoch: 1 [38400/50000 (77%)]	Train Loss: 0.441231
Train Epoch: 1 [44800/50000 (90%)]	Train Loss: 0.919078

[EPOCH: 1], 	Test Loss: 0.0255, 	Test Accuracy: 70.41 % 

Train Epoch: 2 [0/50000 (0%)]	Train Loss: 0.807751
Train Epoch: 2 [6400/50000 (13%)]	Train Loss: 0.987507
Train Epoch: 2 [12800/50000 (26%)]	Train Loss: 0.686738
Train Epoch: 2 [19200/50000 (38%)]	Train Loss: 0.773692
Train Epoch: 2 [25600/50000 (51%)]	Train Loss: 0.452768
Train Epoch: 2 [32000/50000 (64%)]	Train Loss: 0.751910
Train Epoch: 2 [38400/50000 (77%)]	Train Loss: 0.545849
Train Epoch: 2 [44800/50000 (90%)]	Train Loss: 0.521982

[EPOCH: 2], 	Test Loss: 0.0192, 	Test Accuracy: 80.07 % 